In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
from bs4 import BeautifulSoup
import urllib2
import requests

In [3]:
import json
with open("candidate_pages.json") as json_file:
    polinfo = json.load(json_file)
    json_file.close()

In [4]:
polinfo.keys()[0:2]

[u'http://myneta.info/ls2009/candidate.php?candidate_id=7831',
 u'http://myneta.info/ls2009/candidate.php?candidate_id=1245']

In [78]:
#polinfo['http://myneta.info/ls2009/candidate.php?candidate_id=7831']
len('http://myneta.info/loksabha')
s='http://myneta.info/loksabha2004/'
s[27:31]
print s[19:21]
v='http://myneta.info/ls2014/'
print v[19:25]

lo
ls2014


In [63]:
#TEST LS 2004 or LS 2009: MOVABLE ASSETS
soup1=BeautifulSoup(polinfo['http://myneta.info/ls2009/candidate.php?candidate_id=7831'],"html.parser")
table1 = soup1.find("a", attrs={"name": "movable_assets"}).find("table")
for r in table1.findAll('td'):
    if r.get_text()=="iv":
        x = r.findAllNext('b',limit=4)[3].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
x

'6,00,000'

In [71]:
#TEST LS 2014: : MOVABLE ASSETS
soup2=BeautifulSoup(polinfo['http://myneta.info/ls2014/candidate.php?candidate_id=4062'],"html.parser")
table2 = soup2.find("a", attrs={"name": "movable_assets"}).find("table")
for r in table2.findAll('td'):
    if r.get_text()=="Totals (Calculated as Sum of Values)":
        y = r.findAllNext('b',limit=6)[5].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
y

'73,88,000'

In [96]:
def movable_assets(info,url):
    soup=BeautifulSoup(info[url],"html.parser")
    table = soup.find("a", attrs={"name": "movable_assets"}).find("table")
    madict={}
    madict["url"]=url
    
    if url[19:21]=="lo":
        madict['year']=url[len('http://myneta.info/loksabha'): 4 + len('http://myneta.info/loksabha')]
    else:
        madict['year']=url[len('http://myneta.info/ls'): 4 + len('http://myneta.info/ls')]
   
    for r in table.findAll('td'):
        try:
            if r.get_text()=="i":
                madict["Cash"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="ii":
                madict["Deposits in Banks, Financial Institutions and Non-Banking Financial Companies"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

            if r.get_text()=="iii":
                madict["Bonds, Debentures and Shares in companies"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()
                             # use different tag and attribute to pick up text. Limit specifies # of iteration for findall method
            if r.get_text()=="iv":
                madict["NSS, Postal Savings etc"] = r.findAllNext('b',limit=4)[1].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
                madict["LIC or other insurance Policies"] = r.findAllNext('b',limit=4)[3].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
           
            
            if url[19:25]=="ls2014":
                
                if r.get_text()=="v":
                    madict["Personal loans/advance given"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vi":
                    madict["Motor Vehicles (details of make, etc.)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vii":
                    madict["Jewellery (give details weight value)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="viii":
                    madict["Other assets, such as values of claims / interests"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="Gross Total Value (as per Affidavit)":
                    madict["Gross Total Value (as per Affidavit)"] = r.findAllNext('b',limit=1)[0].get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="Totals (Calculated as Sum of Values)":
                    madict["Totals (Calculated as Sum of Values)"] = r.findAllNext('b',limit=6)[5].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
                    
                madict["TOTAL_MOVABLE_ASSESTS"]=  int(madict["Totals (Calculated as Sum of Values)"].replace(',','') )
      
            else:
                
                if r.get_text()=="v":
                    madict["Motor Vehicles (details of make, etc.)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vi":
                    madict["Jewellery (give details weight value)"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="vii":
                    madict["Other assets, such as values of claims / interests"] = r.findNext('b').get_text().encode('ascii', 'ignore').replace("Rs","").strip()

                if r.get_text()=="Totals":
                    madict["Totals (Calculated as Sum of Values)"] = r.findAllNext('b', limit=6)[5].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
        
                if madict["LIC or other insurance Policies"]=="Nil":
                    madict["TOTAL_MOVABLE_ASSESTS"]=  int(madict["Totals (Calculated as Sum of Values)"].replace(',',''))
                else:
                    madict["TOTAL_MOVABLE_ASSESTS"]=  int(madict["Totals (Calculated as Sum of Values)"].replace(',','')) + int(madict["LIC or other insurance Policies"].replace(',',''))
        
        except:
            pass    
        

    return madict


In [94]:
movable_assets(polinfo,'http://myneta.info/ls2009/candidate.php?candidate_id=7831')

{'Bonds, Debentures and Shares in companies': 'Nil',
 'Cash': '50,000',
 'Deposits in Banks, Financial Institutions and Non-Banking Financial Companies': '20,000',
 'Jewellery (give details weight value)': '1,50,000',
 'LIC or other insurance Policies': '6,00,000',
 'Motor Vehicles (details of make, etc.)': '0',
 'NSS, Postal Savings etc': 'Nil',
 'Other assets, such as values of claims / interests': 'Nil',
 'TOTAL_MOVABLE_ASSESTS': 820000,
 'Totals (Calculated as Sum of Values)': '2,20,000',
 'url': 'http://myneta.info/ls2009/candidate.php?candidate_id=7831',
 'year': '2009'}

In [95]:
movable_assets(polinfo,'http://myneta.info/ls2014/candidate.php?candidate_id=4062')

{'Bonds, Debentures and Shares in companies': 'Nil',
 'Cash': '7,00,000',
 'Deposits in Banks, Financial Institutions and Non-Banking Financial Companies': '3,88,000',
 'Gross Total Value (as per Affidavit)': '73,88,000',
 'Jewellery (give details weight value)': '18,00,000',
 'LIC or other insurance Policies': 'Nil',
 'Motor Vehicles (details of make, etc.)': '13,00,000',
 'NSS, Postal Savings etc': 'Nil',
 'Other assets, such as values of claims / interests': 'Nil',
 'Personal loans/advance given': '32,00,000',
 'TOTAL_MOVABLE_ASSESTS': 7388000,
 'Totals (Calculated as Sum of Values)': '73,88,000',
 'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=4062',
 'year': '2014'}

In [ ]:
#TEST LS 2004 or LS 2009: IMMOVABLE ASSETS
soup1=BeautifulSoup(polinfo['http://myneta.info/ls2009/candidate.php?candidate_id=7831'],"html.parser")
table1 = soup1.findAll("table",attrs={"class":"table1"})[2]
for r in table1.findAll('td'):
    if r.get_text()=="iv":
        x = r.findAllNext('b',limit=4)[3].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
x

In [ ]:
#TEST LS 2014: : IMMOVABLE ASSETS
soup2=BeautifulSoup(polinfo['http://myneta.info/ls2014/candidate.php?candidate_id=4062'],"html.parser")
table2 = soup2.findAll("table",attrs={"class":"table1"})[2]
for r in table2.findAll('td'):
    if r.get_text()=="Totals (Calculated as Sum of Values)":
        y = r.findAllNext('b',limit=6)[5].get_text().encode('ascii', 'ignore').replace("Rs","").strip()
y

In [101]:
immovable_assets(polinfo,'http://myneta.info/ls2009/candidate.php?candidate_id=7831')

{'Agricultural Land': '0',
 'Commercial Buildings': 'Nil',
 'Non Agricultural Land': 'Nil',
 'Others': 'Nil',
 'Residential Buildings': '5,00,000',
 'TOTAL_IMMOVABLE_ASSESTS': '5,00,000',
 'url': 'http://myneta.info/ls2009/candidate.php?candidate_id=7831',
 'year': '2009'}

In [102]:
immovable_assets(polinfo,'http://myneta.info/ls2014/candidate.php?candidate_id=4062')

{'Agricultural Land': '5,50,000',
 'Commercial Buildings': 'Nil',
 'Non Agricultural Land': '4,00,00,000',
 'Others': 'Nil',
 'Residential Buildings': '1,00,00,000',
 'TOTAL_IMMOVABLE_ASSESTS': '5,05,50,000',
 'Total Current Market Value of (i) to (v) (as per Affidavit)': '5,05,50,000',
 'url': 'http://myneta.info/ls2014/candidate.php?candidate_id=4062',
 'year': '2014'}

In [103]:
def immovable_assets(info,url):
    soup=BeautifulSoup(info[url],"html.parser")
    table = soup.findAll("table",attrs={"class":"table1"})[2]
    immadict={}
    immadict["url"]=url
    
    if url[19:21]=="lo":
        immadict['year']=url[len('http://myneta.info/loksabha'): 4 + len('http://myneta.info/loksabha')]
    else:
        immadict['year']=url[len('http://myneta.info/ls'): 4 + len('http://myneta.info/ls')]

    for r in table.findAll('td'):
        try:
            if r.get_text()=="i":
                immadict["Agricultural Land"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="ii":
                immadict["Non Agricultural Land"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")
           
            if r.get_text()=="iii":
                immadict["Commercial Buildings"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="iv":
                immadict["Residential Buildings"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")

            if r.get_text()=="v":
                immadict["Others"] = r.findNext('b').get_text().strip().encode('ascii', 'ignore').replace("Rs","")
            
            if url[19:25]=="ls2014":
                
                if r.get_text()=="Total Current Market Value of (i) to (v) (as per Affidavit)":
                    immadict["Total Current Market Value of (i) to (v) (as per Affidavit)"] = r.findAllNext('b',limit=1)[0].get_text().strip().encode('ascii', 'ignore').replace("Rs","")

                if r.get_text()=="Totals Calculated":
                    immadict["TOTAL_IMMOVABLE_ASSESTS"] = r.findAllNext('b',limit=6)[5].get_text().strip().encode('ascii', 'ignore').replace("Rs","")
            else:
                if r.get_text()=="Totals":
                    immadict["TOTAL_IMMOVABLE_ASSESTS"] = r.findAllNext('b',limit=6)[5].get_text().strip().encode('ascii', 'ignore').replace("Rs","")
                
        except:
            pass
    
    return immadict
                

In [100]:
def cleaner(d):
    for key in d.keys():
        if d[key]=='Nil':
            d[key]==np.nan
        else:
            try:
                d[key]=float(d[key].replace(',',''))
            except:
                pass
    return d

In [97]:
%%time
movassets={}
for url in polinfo.keys():
    try:
        massetdict=cleaner(movable_assets(polinfo,url))
        movassets[url] = massetdict
        
    except TypeError: 
        print "error: ",url

CPU times: user 34min 11s, sys: 18.3 s, total: 34min 30s
Wall time: 38min 42s


In [13]:
#fd=open("movassets.json","w")
#json.dump(movassets, fd)
#fd.close()

In [104]:
%%time
immovassets={}
for url in polinfo.keys():
    try:
        immassetdict=cleaner(immovable_assets(polinfo,url))
        immovassets[url] = immassetdict
        
    except TypeError: 
        print "error: ",url
        

CPU times: user 36min 40s, sys: 21 s, total: 37min 1s
Wall time: 40min 6s


In [105]:
len(movassets)

19134

In [106]:
movassets=pd.DataFrame(movassets)
movassets=movassets.transpose()

In [107]:
movassets=movassets.replace('Nil',np.nan)
movassets.head(2)

,"Bonds, Debentures and Shares in companies",Cash,"Deposits in Banks, Financial Institutions and Non-Banking Financial Companies",Gross Total Value (as per Affidavit),Jewellery (give details weight value),LIC or other insurance Policies,"Motor Vehicles (details of make, etc.)","NSS, Postal Savings etc","Other assets, such as values of claims / interests",Personal loans/advance given,TOTAL_MOVABLE_ASSESTS,Totals (Calculated as Sum of Values),url,year
http://myneta.info/loksabha2004/candidate.php?candidate_id=1,NaN,70500,1519565,NaN,128501,350000,410000,316155,621439,NaN,3416160,3066160,http://myneta.info/loksabha2004/candidate.php?...,2004
http://myneta.info/loksabha2004/candidate.php?candidate_id=10,NaN,70000,NaN,NaN,0,450000,0,NaN,0,NaN,520000,70000,http://myneta.info/loksabha2004/candidate.php?...,2004


In [108]:
movassets.to_csv("movassets.csv", header=True, index=True, encoding='utf-8')

In [109]:
immovassets=pd.DataFrame(immovassets)
immovassets=immovassets.transpose()

In [110]:
immovassets=immovassets.replace('Nil',np.nan)
immovassets.head(2)

,Agricultural Land,Commercial Buildings,Non Agricultural Land,Others,Residential Buildings,TOTAL_IMMOVABLE_ASSESTS,Total Current Market Value of (i) to (v) (as per Affidavit),url,year
http://myneta.info/loksabha2004/candidate.php?candidate_id=1,846000,NaN,64580,NaN,1950000,2860580,NaN,http://myneta.info/loksabha2004/candidate.php?...,2004
http://myneta.info/loksabha2004/candidate.php?candidate_id=10,3000000,NaN,NaN,NaN,NaN,3000000,NaN,http://myneta.info/loksabha2004/candidate.php?...,2004


In [111]:
immovassets.to_csv("immovassets.csv", header=True, index=True, encoding='utf-8')

In [112]:
# example:
#dft=pd.DataFrame(massets[polinfo.keys()[0]])
#dft.set_index('srn',inplace=True)
#dft.head()

In [113]:
import gc
gc.collect()

301